In [1]:
import ot
import pandas as pd
import numpy as np
import os

In [2]:
def kl_divergence_backend(X, Y):
    """
    Returns pairwise KL divergence (over all pairs of samples) of two matrices X and Y.
    
    Takes advantage of POT backend to speed up computation.
    
    Args:
        X: np array with dim (n_samples by n_features)
        Y: np array with dim (m_samples by n_features)
    
    Returns:
        D: np array with dim (n_samples by m_samples). Pairwise KL divergence matrix.
    """
    assert X.shape[1] == Y.shape[1], "X and Y do not have the same number of features."

    nx = ot.backend.get_backend(X,Y)
    
    X = X/nx.sum(X,axis=1, keepdims=True)
    Y = Y/nx.sum(Y,axis=1, keepdims=True)
    log_X = nx.log(X)
    log_Y = nx.log(Y)
    X_log_X = nx.einsum('ij,ij->i',X,log_X)
    X_log_X = nx.reshape(X_log_X,(1,X_log_X.shape[0]))
    D = X_log_X.T - nx.dot(X,log_Y.T)
    return nx.to_numpy(D)

def f(G):
    return ot.gromov.gwloss(constC, hC1, hC2, G)

def df(G):
    return ot.gromov.gwggrad(constC, hC1, hC2, G)


In [3]:
dissimilarity='euc'
norm=False
alpha=0.8
loss_fun='square_loss'
backend=ot.backend.NumpyBackend()
nx=backend

simulate

In [23]:
dirs="/data02/tguo/space_batch_effect/simulate/gtt_output/coordinate_file/"
batch_sim="_1"
types="_types4"
clusters=pd.read_csv(dirs+"gtt_clusters"+batch_sim+types+".csv",header=0,index_col=0,sep=",")
ub=np.unique(clusters['batch'])
for i in np.arange(1,len(ub)):
    uc=np.intersect1d(clusters['clusters'][clusters['batch']==ub[0]],
                     clusters['clusters'][clusters['batch']==ub[i]])
    for clust in uc:
        embed1=pd.read_csv(dirs+"embed_"+ub[0]+"_"+str(clust)+batch_sim+types+".csv",header=0,index_col=0,sep=",")
        embed2=pd.read_csv(dirs+"embed_"+ub[i]+"_"+str(clust)+batch_sim+types+".csv",header=0,index_col=0,sep=",")
        coord1=pd.read_csv(dirs+"coord_"+ub[0]+"_"+str(clust)+batch_sim+types+".csv",header=0,index_col=0,sep=",")
        coord2=pd.read_csv(dirs+"coord_"+ub[i]+"_"+str(clust)+batch_sim+types+".csv",header=0,index_col=0,sep=",")
        coord1=coord1.loc[embed1.index,:]
        coord2=coord2.loc[embed2.index,:]
        ###每个batch内部spot的空间距离####
        a=np.float64(nx.from_numpy(coord1.values[:,:2]))
        b=np.float64(nx.from_numpy(coord2.values[:,:2]))
        D1=ot.dist(a,a, metric='euclidean')
        D2=ot.dist(b,b, metric='euclidean')
        if norm:
            D1 /= nx.min(D1[D1>0])
            D2 /= nx.min(D2[D2>0])
        ####两个batch spot的低维表示的距离#####
        X1,X2 = nx.from_numpy(embed1.values), nx.from_numpy(embed2.values)
        if dissimilarity.lower()=='euclidean' or dissimilarity.lower()=='euc':
            M = ot.dist(X1,X2)
        else:
            s1 = X1 + 0.01
            s2 = X2 + 0.01
            M = kl_divergence_backend(s1, s2)
            M = nx.from_numpy(M)
        ####每个batch的spot的分布#####
        d1 = nx.ones((embed1.shape[0],))/embed1.shape[0]
        d2 = nx.ones((embed2.shape[0],))/embed2.shape[0]
        ####计算mapping#####
        constC, hC1, hC2 = ot.gromov.init_matrix(D1, D2, d1, d2, loss_fun)
        G0 = d1[:, None] * d2[None, :]
        res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC)
        pi=pd.DataFrame(res,index=embed1.index,columns=embed2.index)
        pi.to_csv(dirs+"gwd_pi_"+ub[0]+"_"+ub[i]+"_"+str(clust)+batch_sim+types+".csv")

DLPFC

In [63]:
alpha=0.8
dirs="/data02/tguo/space_batch_effect/human_DLPFC_10x/gtt_output/coordinate_file/"
samples=np.array(['151507','151508','151509','151510','151669','151670','151671','151672','151673','151674','151675','151676'])
samples=samples[[8,9,10,11]]
flags=""
for i in samples:
    flags=flags+"_"+i
clusters=pd.read_csv(dirs+"gtt_clusters"+flags+".csv",header=0,index_col=0,sep=",")
ub=np.unique(clusters['batch'])
for i in np.arange(1,len(ub)):
    uc=np.intersect1d(clusters['clusters'][clusters['batch']==ub[0]],
                     clusters['clusters'][clusters['batch']==ub[i]])
    for clust in uc:
        embed1=pd.read_csv(dirs+"embed_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        embed2=pd.read_csv(dirs+"embed_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=pd.read_csv(dirs+"coord_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord2=pd.read_csv(dirs+"coord_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=coord1.loc[embed1.index,:]
        coord2=coord2.loc[embed2.index,:]
        ###每个batch内部spot的空间距离####
        a=np.float64(nx.from_numpy(coord1.values[:,:2]))
        b=np.float64(nx.from_numpy(coord2.values[:,:2]))
        D1=ot.dist(a,a, metric='euclidean')
        D2=ot.dist(b,b, metric='euclidean')
        if norm:
            D1 /= nx.min(D1[D1>0])
            D2 /= nx.min(D2[D2>0])
        ####两个batch spot的低维表示的距离#####
        X1,X2 = nx.from_numpy(embed1.values), nx.from_numpy(embed2.values)
        if dissimilarity.lower()=='euclidean' or dissimilarity.lower()=='euc':
            M = ot.dist(X1,X2)
        else:
            s1 = X1 + 0.01
            s2 = X2 + 0.01
            M = kl_divergence_backend(s1, s2)
            M = nx.from_numpy(M)
        ####每个batch的spot的分布#####
        d1 = nx.ones((embed1.shape[0],))/embed1.shape[0]
        d2 = nx.ones((embed2.shape[0],))/embed2.shape[0]
        ####计算mapping#####
        constC, hC1, hC2 = ot.gromov.init_matrix(D1, D2, d1, d2, loss_fun)
        G0 = d1[:, None] * d2[None, :]
        res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC)
        pi=pd.DataFrame(res,index=embed1.index,columns=embed2.index)
        pi.to_csv(dirs+"gwd_pi_"+str(ub[0])+"_"+str(ub[i])+"_"+str(clust)+flags+".csv")

mouse brain

In [7]:
dirs="/data02/tguo/space_batch_effect/mouse_brain/gtt_output/coordinate_file/"
samples=['anterior1','anterior2']
flags=""
for i in samples:
    flags=flags+"_"+i
clusters=pd.read_csv(dirs+"gtt_clusters"+flags+".csv",header=0,index_col=0,sep=",")
ub=np.unique(clusters['batch'])
for i in np.arange(1,len(ub)):
    uc=np.intersect1d(clusters['clusters'][clusters['batch']==ub[0]],
                     clusters['clusters'][clusters['batch']==ub[i]])
    for clust in uc:
        embed1=pd.read_csv(dirs+"embed_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        embed2=pd.read_csv(dirs+"embed_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=pd.read_csv(dirs+"coord_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord2=pd.read_csv(dirs+"coord_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=coord1.loc[embed1.index,:]
        coord2=coord2.loc[embed2.index,:]
        ###每个batch内部spot的空间距离####
        a=np.float64(nx.from_numpy(coord1.values[:,:2]))
        b=np.float64(nx.from_numpy(coord2.values[:,:2]))
        D1=ot.dist(a,a, metric='euclidean')
        D2=ot.dist(b,b, metric='euclidean')
        if norm:
            D1 /= nx.min(D1[D1>0])
            D2 /= nx.min(D2[D2>0])
        ####两个batch spot的低维表示的距离#####
        X1,X2 = nx.from_numpy(embed1.values), nx.from_numpy(embed2.values)
        if dissimilarity.lower()=='euclidean' or dissimilarity.lower()=='euc':
            M = ot.dist(X1,X2)
        else:
            s1 = X1 + 0.01
            s2 = X2 + 0.01
            M = kl_divergence_backend(s1, s2)
            M = nx.from_numpy(M)
        ####每个batch的spot的分布#####
        d1 = nx.ones((embed1.shape[0],))/embed1.shape[0]
        d2 = nx.ones((embed2.shape[0],))/embed2.shape[0]
        ####计算mapping#####
        constC, hC1, hC2 = ot.gromov.init_matrix(D1, D2, d1, d2, loss_fun)
        G0 = d1[:, None] * d2[None, :]
        res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC)
        pi=pd.DataFrame(res,index=embed1.index,columns=embed2.index)
        pi.to_csv(dirs+"gwd_pi_"+str(ub[0])+"_"+str(ub[i])+"_"+str(clust)+flags+".csv")

10x 冠状面

In [10]:
alpha=0.8
dirs="/data02/tguo/space_batch_effect/Hippo/gtt_output/coordinate_file/"
samples=['10X_Normal','10X_DAPI','10X_FFPE']
flags=""
for i in samples:
    flags=flags+"_"+i
clusters=pd.read_csv(dirs+"gtt_clusters"+flags+".csv",header=0,index_col=0,sep=",")
ub=np.unique(clusters['batch'])
ub=['10X_Normal','10X_DAPI','10X_FFPE']
for i in np.arange(1,len(ub)):
    uc=np.intersect1d(clusters['clusters'][clusters['batch']==ub[0]],
                     clusters['clusters'][clusters['batch']==ub[i]])
    for clust in uc:
        embed1=pd.read_csv(dirs+"embed_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        embed2=pd.read_csv(dirs+"embed_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=pd.read_csv(dirs+"coord_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord2=pd.read_csv(dirs+"coord_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=coord1.loc[embed1.index,:]
        coord2=coord2.loc[embed2.index,:]
        ###每个batch内部spot的空间距离####
        a=np.float64(nx.from_numpy(coord1.values[:,:2]))
        b=np.float64(nx.from_numpy(coord2.values[:,:2]))
        D1=ot.dist(a,a, metric='euclidean')
        D2=ot.dist(b,b, metric='euclidean')
        if norm:
            D1 /= nx.min(D1[D1>0])
            D2 /= nx.min(D2[D2>0])
        ####两个batch spot的低维表示的距离#####
        X1,X2 = nx.from_numpy(embed1.values), nx.from_numpy(embed2.values)
        if dissimilarity.lower()=='euclidean' or dissimilarity.lower()=='euc':
            M = ot.dist(X1,X2)
        else:
            s1 = X1 + 0.01
            s2 = X2 + 0.01
            M = kl_divergence_backend(s1, s2)
            M = nx.from_numpy(M)
        ####每个batch的spot的分布#####
        d1 = nx.ones((embed1.shape[0],))/embed1.shape[0]
        d2 = nx.ones((embed2.shape[0],))/embed2.shape[0]
        ####计算mapping#####
        constC, hC1, hC2 = ot.gromov.init_matrix(D1, D2, d1, d2, loss_fun)
        G0 = d1[:, None] * d2[None, :]
        res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC)
        pi=pd.DataFrame(res,index=embed1.index,columns=embed2.index)
        pi.to_csv(dirs+"gwd_pi_"+str(ub[0])+"_"+str(ub[i])+"_"+str(clust)+flags+".csv")

mouse ob

In [18]:
alpha=0.1
dirs="/data02/tguo/space_batch_effect/mouse_OB/gtt_output/coordinate_file/"
samples=['BGI','SlideV2','10X']
# samples=['BGI','SlideV2','scRNA']
flags=""
for i in samples:
    flags=flags+"_"+i
clusters=pd.read_csv(dirs+"gtt_clusters"+flags+".csv",header=0,index_col=0,sep=",")
ub=np.unique(clusters['batch'])
ub=samples
for i in np.arange(1,len(ub)):
    uc=np.intersect1d(clusters['clusters'][clusters['batch']==ub[0]],
                     clusters['clusters'][clusters['batch']==ub[i]])
    for clust in uc:
        embed1=pd.read_csv(dirs+"embed_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        embed2=pd.read_csv(dirs+"embed_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=pd.read_csv(dirs+"coord_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord2=pd.read_csv(dirs+"coord_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
        coord1=coord1.loc[embed1.index,:]
        coord2=coord2.loc[embed2.index,:]
        ###每个batch内部spot的空间距离####
#         a=np.float64(nx.from_numpy(coord1.values[:,:2]))
#         b=np.float64(nx.from_numpy(coord2.values[:,:2]))
        a=np.float64(nx.from_numpy(coord1.values))
        b=np.float64(nx.from_numpy(coord2.values))
        D1=ot.dist(a,a, metric='euclidean')
        D2=ot.dist(b,b, metric='euclidean')
        if norm:
            D1 /= nx.min(D1[D1>0])
            D2 /= nx.min(D2[D2>0])
        ####两个batch spot的低维表示的距离#####
        X1,X2 = nx.from_numpy(embed1.values), nx.from_numpy(embed2.values)
        if dissimilarity.lower()=='euclidean' or dissimilarity.lower()=='euc':
            M = ot.dist(X1,X2)
        else:
            s1 = X1 + 0.01
            s2 = X2 + 0.01
            M = kl_divergence_backend(s1, s2)
            M = nx.from_numpy(M)
        ####每个batch的spot的分布#####
        d1 = nx.ones((embed1.shape[0],))/embed1.shape[0]
        d2 = nx.ones((embed2.shape[0],))/embed2.shape[0]
        ####计算mapping#####
        constC, hC1, hC2 = ot.gromov.init_matrix(D1, D2, d1, d2, loss_fun)
        G0 = d1[:, None] * d2[None, :]
        res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC,numItermax=100000,numItermaxEmd=1e6)
        pi=pd.DataFrame(res,index=embed1.index,columns=embed2.index)
        pi.to_csv(dirs+"gwd_pi_"+str(ub[0])+"_"+str(ub[i])+"_"+str(clust)+flags+".csv")

In [16]:
coord2

,GTT_4,GTT_5,GTT_6,GTT_7,GTT_8,GTT_9,GTT_10,GTT_11,GTT_12,GTT_13,...,GTT_22,GTT_23,GTT_24,GTT_25,GTT_26,GTT_27,GTT_28,GTT_29,GTT_30,GTT_31
SlideV2-AAAAAGATCAGGTT,-0.170360,-0.153913,-0.139782,0.188074,0.109938,0.491718,-0.157277,0.179078,-0.620951,0.128879,...,-0.377249,-0.152204,-0.101502,0.468256,0.475027,-0.438562,0.155265,0.160116,0.675001,0.451747
SlideV2-AAAAAGGTTTTGGG,0.081309,0.384900,-0.293209,0.188151,-0.338261,-0.274472,0.199760,-0.136510,0.187796,0.287287,...,0.061433,0.043816,-0.238022,0.010100,0.052646,-0.206263,0.071546,-0.026444,-0.211838,-0.302042
SlideV2-AAAAATCGTCGCAC,-0.452554,0.560075,0.160644,0.703167,0.163454,-0.084037,-0.249380,0.158641,0.478881,0.029622,...,0.082859,-0.536301,-0.126102,-0.071094,0.604978,0.045158,-0.103886,-0.058465,0.364410,0.265119
SlideV2-AAAAATTCTAAGTA,0.030323,0.218073,0.254941,0.125699,-0.161013,-0.191530,-0.416796,0.041352,0.127006,0.376724,...,0.205280,0.134790,0.059280,0.129645,0.185062,-0.098025,0.310435,-0.019416,0.155625,0.264185
SlideV2-AAAAATTCTGAACC,-0.293104,0.100765,0.073194,0.488853,0.268205,-0.226572,0.405138,-0.011226,0.043004,0.139730,...,0.087892,0.215490,0.371550,-0.204795,0.323758,-0.133126,0.294960,0.139621,0.394841,0.688287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SlideV2-TTTTTGAATGTCAA,-0.400190,0.362623,-0.098748,0.354262,0.098753,0.297566,-0.086875,-0.207666,0.321166,0.041881,...,-0.040839,-0.104181,0.350871,-0.128532,-0.089945,-0.038234,-0.153584,0.160341,-0.052390,-0.195149
SlideV2-TTTTTTATTTTTTT,0.081960,0.215202,0.062646,0.142394,0.152964,-0.084917,0.387747,0.016497,-0.500035,0.107720,...,-0.084059,0.192300,0.234828,-0.300199,0.098774,-0.389207,-0.302599,0.013265,-0.015795,0.000787
SlideV2-TTTTTTTTGCTAAC,0.223886,0.409774,0.010514,0.750321,-0.145308,-0.057763,0.163303,-0.199252,-0.155199,0.283754,...,-0.257665,0.204354,0.289818,-0.185497,0.543484,-0.279441,-0.060972,0.226209,0.360027,-0.038424
SlideV2-TTTTTTTTTATACT,-0.240303,-0.026589,0.151231,-0.271260,0.101614,0.010893,-0.368306,0.129474,0.600429,-0.037006,...,-0.071060,0.312544,-0.571222,0.023424,-0.073096,0.247364,0.197792,0.014575,-1.198060,-0.281605


In [19]:
clust=0
i=1
embed1=pd.read_csv(dirs+"embed_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
embed2=pd.read_csv(dirs+"embed_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
coord1=pd.read_csv(dirs+"coord_"+str(ub[0])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
coord2=pd.read_csv(dirs+"coord_"+str(ub[i])+"_"+str(clust)+flags+".csv",header=0,index_col=0,sep=",")
coord1=coord1.loc[embed1.index,:]
coord2=coord2.loc[embed2.index,:]
###每个batch内部spot的空间距离####
a=np.float64(nx.from_numpy(coord1.values[:,:2]))
b=np.float64(nx.from_numpy(coord2.values[:,:2]))
D1=ot.dist(a,a, metric='euclidean')
D2=ot.dist(b,b, metric='euclidean')
if norm:
    D1 /= nx.min(D1[D1>0])
    D2 /= nx.min(D2[D2>0])
####两个batch spot的低维表示的距离#####
X1,X2 = nx.from_numpy(embed1.values), nx.from_numpy(embed2.values)
if dissimilarity.lower()=='euclidean' or dissimilarity.lower()=='euc':
    M = ot.dist(X1,X2)
else:
    s1 = X1 + 0.01
    s2 = X2 + 0.01
    M = kl_divergence_backend(s1, s2)
    M = nx.from_numpy(M)
####每个batch的spot的分布#####
d1 = nx.ones((embed1.shape[0],))/embed1.shape[0]
d2 = nx.ones((embed2.shape[0],))/embed2.shape[0]
# ####计算mapping#####
constC, hC1, hC2 = ot.gromov.init_matrix(D1, D2, d1, d2, loss_fun)
G0 = d1[:, None] * d2[None, :]
# res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC)
# pi=pd.DataFrame(res,index=embed1.index,columns=embed2.index)

In [36]:
alpha=0.5
res=ot.gromov.cg(d1, d2, (1 - alpha) * M, alpha, f, df, G0, armijo=False, C1=D1, C2=D2, constC=constC,numItermax=100000,numItermaxEmd=1e6)

In [39]:
np.where(np.sum(res,axis=1)==0)

(array([], dtype=int64),)